In [1]:
import os
from langchain_ollama import ChatOllama
from langchain_ollama import OllamaLLM
from src.data.preprocess_data import preprocess_data
from src.utils.param_extractor import extract_params
from src.llm.llm_client import llm, llm_raw
from src.llm import workflow_manager

import time
import threading
import uvicorn
from src.llm import fastapi_client
from src.utils.logger import setup_logger, get_logger

import pandas as pd
from pandasai import SmartDataframe

from src.utils.backend_duckdb import DSI_Duck

/vast/projects/exasky/justintam/project_ollama/genericio/legacy_python


In [2]:
from langchain.agents import initialize_agent, Tool, AgentType

In [3]:
def start_fastapi():
    uvicorn.run("src.llm.fastapi_server:app", host="127.0.0.1", port=8000, reload=False, log_level="info")

In [4]:
# Setup logger
ENABLE_LOGGING = False
ENABLE_DEBUG = False
setup_logger(ENABLE_LOGGING, ENABLE_DEBUG)
logging = get_logger(__name__)

# Start FastAPI server in background
server_thread = threading.Thread(target=start_fastapi, daemon=True)
server_thread.start()
logging.info("[INFO] FastAPI server running at http://127.0.0.1:8000")
print("[INFO] Sandbox started.")

params = {'task': 'find_largest_object', 'object_type': 'halo', 'timestep': 498, 'n': 10}

# Run workflow based on task classification and print result
result = workflow_manager.run_workflow(params["task"], params)
# logging.info(f"[WORKFLOW OUTPUT] {result}")
print(f"[Workflow output] {result.info()}")

print(type(result))

[INFO] Sandbox started.


INFO:     Started server process [663636]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 148789 to 232768
Columns: 143 entries, fof_halo_count to sod_halo_core_cusp
dtypes: float64(143)
memory usage: 11.2 KB
[Workflow output] None
<class 'pandas.core.frame.DataFrame'>


In [5]:
user_query = "Which row has the highest fof_halo_count??"

In [6]:
duckdb = DSI_Duck()
duckdb.connect_to_db("test_db.duckdb")
duckdb.import_dataframe(result, "example_table")
duckdb.get_tables_info()

Data inserted into existing table 'example_table'.


[['example_table', 143, 20]]

In [7]:
# Define tools for the agent
tools = [
    Tool(
        name="DuckDB",
        func=lambda query: conn.execute(query).fetchall(),
        description="Use this tool to execute SQL queries on the DuckDB database."
    )
]

# Initialize the agent
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

/tmp/ipykernel_663636/3985517837.py:11: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [8]:
# Define the system prompt
system_prompt = f"""
You are a data analyst for a retail company working with a Teradata system.

1. Users send you business questions in plain English, and you provide answers to those questions based on the data in the provided databases.
2. To generate answers, you must construct an SQL statement to query the database.
   - The SQL query must be written as a single line of text without carriage returns or line breaks.
   - Ensure that the query adheres to DuckDB's SQL dialect and does not include unsupported keywords such as `LIMIT`.
   - Joins across tables should be used when necessary to fulfill the user's request.
3. Execute the SQL query in DuckDB.
4. Present the query results to the user in plain English.
"""



In [9]:
import requests

OLLAMA_API_URL = "http://localhost:11434/api/generate"
MODEL_NAME = "mistral-small3.1:latest"  # Or any model you've pulled

def ask_ollama(prompt: str) -> str:
    payload = {
        "model": MODEL_NAME,
        "prompt": prompt,
        "stream": False  # Set to True for streaming responses
    }

    response = requests.post(OLLAMA_API_URL, json=payload)
    if response.status_code == 200:
        return response.json().get("response", "").strip()
    else:
        raise Exception(f"Ollama API error: {response.status_code} - {response.text}")

ask_ollama("Hello")

ConnectionError: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/generate (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x14e375f81790>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [ ]:
while True:
    query = input("You: ")
    if query.lower() in {"exit", "quit"}:
        break
    try:
        response = llm.invoke(query)
        print("Ollama: ", response)
        messages.append(response)

    except Exception as e:
        print("Error:", e)

Error: [Errno 111] Connection refused
Error: [Errno 111] Connection refused
Error: [Errno 111] Connection refused


In [ ]:
# Define a function to handle user queries
def handle_query(user_input):
    response = agent.run(user_input)
    return response

# Example usage
user_input = "What is the average salary in the example_table?"
result = handle_query(user_input)
print(result)

/tmp/ipykernel_615657/2279628751.py:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent.run(user_input)




> Entering new AgentExecutor chain...


ConnectError: [Errno 111] Connection refused